In [3]:
import pandas as pd
ess = pd.read_csv('ess.csv', low_memory=False)
print(ess.shape)

(44387, 534)


In [4]:
print(ess.loc[:,'happy'].head())

0    5
1    5
2    8
3    8
4    5
Name: happy, dtype: int64


In [5]:
print(ess.loc[:,'sclmeet'].head())

0    6
1    4
2    4
3    4
4    6
Name: sclmeet, dtype: int64


In [6]:
ess = ess.loc[ess['sclmeet'] <= 10,:].copy()
ess = ess.loc[ess['rlgdgr'] <= 10,:].copy()
ess = ess.loc[ess['hhmmb'] <= 50,:].copy()
ess = ess.loc[ess['netusoft'] <= 5,:].copy()
ess = ess.loc[ess['agea'] <= 200,:].copy()
ess = ess.loc[ess['health'] <= 5,:].copy()
ess = ess.loc[ess['happy'] <= 10,:].copy()
ess = ess.loc[ess['eduyrs'] <= 100,:].copy().reset_index(drop=True)

In [7]:
import numpy as np

social = list(ess.loc[:,'sclmeet'])
happy = list(ess.loc[:,'happy'])
low_social_happiness = [hap for soc,hap in zip(social,happy) if soc <= 5]
high_social_happiness = [hap for soc,hap in zip(social,happy) if soc > 5]
meanlower = np.mean(low_social_happiness)
meanhigher = np.mean(high_social_happiness)

In [8]:
lowererrors = [abs(lowhappy - meanlower) for lowhappy in low_social_happiness]
highererrors = [abs(highhappy - meanhigher) for highhappy in high_social_happiness]
total_error = sum(lowererrors) + sum(highererrors)

In [9]:
def get_splitpoint(allvalues, predictedvalues):
    """
    Функция, находящая лучшую точку разбиения переменной
    для ветвления в дереве принятия решений
    """
    lowest_error = float('inf')
    best_split = None
    best_lowermean = np.mean(predictedvalues)
    best_highermean = np.mean(predictedvalues)
    
    for pctl in range(0,100):
        split_candidate = np.percentile(allvalues, pctl)
        loweroutcomes = [outcome for value, outcome in zip(allvalues, predictedvalues) if value <= split_candidate]
        higheroutcomes = [outcome for value, outcome in zip(allvalues, predictedvalues) if value > split_candidate]
        
        if np.min([len(loweroutcomes), len(higheroutcomes)]) > 0:
            meanlower = np.mean(loweroutcomes)
            meanhigher = np.mean(higheroutcomes)
        
    lowererrors = [abs(outcome - meanlower) for outcome in loweroutcomes]
    highererrors = [abs(outcome - meanhigher) for outcome in higheroutcomes]
    total_error = sum(lowererrors) + sum(highererrors)
    
    if total_error < lowest_error:
        best_split = split_candidate
        lowest_error = total_error
        best_lowermean = meanlower
        best_highermean = meanhigher
        
    return best_split, lowest_error, best_lowermean, best_highermean

In [12]:
allvalues = list(ess.loc[:,'hhmmb'])
predictedvalues = list(ess.loc[:,'happy'])
print(get_splitpoint(allvalues,predictedvalues))

(np.float64(7.0), np.float64(62243.62061562566), np.float64(7.451073100635053), np.float64(7.9890710382513666))


In [30]:
def getsplit(data, variables, outcome_variable):
    """
    Функция перебирает все переменные 
    и находит лучшую переменную для разбиения
    """
    best_var = ''
    lowest_error = float('inf')
    best_split = None
    predictedvalues = list(data.loc[:, outcome_variable])
    best_lowermean = -1
    best_highermean = -1
    
    for var in variables:
        allvalues = list(data.loc[:, var])
        splitted = get_splitpoint(allvalues, predictedvalues)
        
        if splitted[1] < lowest_error:
            best_split = splitted[0]
            lowest_error = splitted[1]
            best_var = var
            best_lowermean = splitted[2]
            best_highermean = splitted[3]
            
    generated_tree = [[best_var, float('-inf'), best_split, best_lowermean], [best_var, best_split, float('inf'), best_highermean]]
    return generated_tree

In [31]:
variables = ['rlgdgr', 'hhmmb', 'netusoft', 'agea', 'eduyrs']
outcome_variable = 'happy'

print(getsplit(ess, variables, outcome_variable))

[['hhmmb', -inf, np.float64(7.0), np.float64(7.451073100635053)], ['hhmmb', np.float64(7.0), inf, np.float64(7.9890710382513666)]]
